In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: sahaana (use `wandb login --relogin` to force relogin)


True

# Imports 

In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from transformers import AutoTokenizer, DistilBertModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [5]:
import sys
#sys.path.append('/lfs/1/sahaana/enrichment/enrich/utils')
sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
 
    
from embedding_datasets import DeepMatcherDataset, EmberEvalDataset
from embedding_models import TripletSingleBERTModel
from embedding_utils import param_header, tokenize_batch  
from embedding_runner import train_model, eval_model
#from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   
#from models import BatchedTripletSingleTowerModel, BatchedTripletSingleBERTModel
#from model_runner import train_model, eval_model
from knn_utils import FaissKNeighbors, knn_top_1_PRFS #, knn_matching_accuracy, find_perfect_recall

In [6]:
%load_ext autoreload
%autoreload 2

# BUMBUM

In [7]:
datasets = {0:"abt_buy_exp_data", 
            1:"amazon_google_exp_data", 
            2:"beer_exp_data", 
            3:"company_exp_data", 
            4:"dblp_acm_exp_data", 
            5:"dblp_scholar_exp_data", 
            6:"dirty_dblp_acm_exp_data", 
            7:"dirty_dblp_scholar_exp_data", 
            8:"dirty_itunes_amazon_exp_data", 
            9:"dirty_walmart_amazon_exp_data", 
            10:"fodors_zagat_exp_data", 
            11:"itunes_amazon_exp_data", 
            12:"walmart_amazon_exp_data"}

In [8]:
for d in datasets:
    print(datasets[d])
    left = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/tableA_processed.pkl'
    right = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/tableB_processed.pkl'

    left = pd.read_pickle(left)
    right = pd.read_pickle(right)

    train_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/train_updated.csv'
    train_df = pd.read_csv(train_df)

    val_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/val_updated.csv'
    val_df = pd.read_csv(val_df)

    test_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/test_updated.csv'
    test_df = pd.read_csv(test_df)

    epochs = 1
    batch_size = 8
    final_size = 200
    lr = .00001
    tl_margin = 1.0
    tl_p = 2
    pool_type = "CLS"
    column = "merged_all"
    shuffle = True
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    tokenizer_max_length = 512
    compute_val = False
    train_size = int(len(train_df)/2)

    bert_path=f'/lfs/1/sahaana/enrichment/ember/pretraining/models/{datasets[d]}-uncased-masked-ALL-BM25'
    bert_model = DistilBertModel.from_pretrained(bert_path, return_dict=True)

    model_name = f'{datasets[d]}-uncased-masked-ALL-BM25-{train_size}'

    train_data = DataLoader(DeepMatcherDataset(left, right, train_size, column, train_df), 
                            batch_size=batch_size,
                            shuffle = shuffle
                            )

    val_data = DataLoader(DeepMatcherDataset(left, right, len(val_df), column, val_df), 
                           batch_size=batch_size,
                           shuffle = False
                          )

    triplet_loss = nn.TripletMarginLoss(margin=tl_margin, p=tl_p)
    losses = []
    val_losses = []
    model = TripletSingleBERTModel(final_size, pool_type, bert_path)
    optimizer = optim.AdamW(model.parameters(), lr=lr)#optim.SGD(model.parameters(), lr=lr)

    save_dir = param_header(batch_size, final_size, lr, pool_type, epochs, train_size)
    save_dir = f'models/{model_name}/{save_dir}/'

    wandb.init(project=model_name)

    train_model(model, 
                tokenizer, 
                tokenize_batch, 
                train_data, 
                val_data, 
                triplet_loss, 
                optimizer, 
                epochs, 
                losses, 
                val_losses, 
                save_dir, 
                compute_val, 
                tokenizer_max_length = tokenizer_max_length)


    left_data = DataLoader(EmberEvalDataset(left, column), 
                           batch_size=batch_size,
                           shuffle = False
                          )
    right_data = DataLoader(EmberEvalDataset(right, column), 
                           batch_size=batch_size,
                           shuffle = False
                          )

    left_embeddings = eval_model(model, tokenizer, left_data, tokenizer_max_length=512)
    right_embeddings = eval_model(model, tokenizer, right_data, tokenizer_max_length=512)

    knn = FaissKNeighbors(k=30)
    knn.fit(right_embeddings)
    neib = knn.kneighbors(left_embeddings)
    print(datasets[d])
    print(f"precision, recall, F1, support: {knn_top_1_PRFS(neib[0], neib[1], test_df)}")
    print()
    print()
    print()

abt_buy_exp_data


wandb: wandb version 0.10.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


abt_buy_exp_data
precision, recall, F1, support: (0.9344262295081968, 0.8300970873786407, 0.8791773778920309, None)



amazon_google_exp_data


Epoch,0
_step,359
_runtime,55
_timestamp,1612991289
train batch loss,0.04524


Epoch,▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train batch loss,█▅█▅▆▆▅▃▆▄▃▂▄▃▂▄▃▂▂▃▂▂▁▁▂▂▁▁▂▁▁▁▂▁▁▁▁▁▁▂


...

itunes_amazon_exp_data
precision, recall, F1, support: (1.0, 0.37037037037037035, 0.5405405405405406, None)



walmart_amazon_exp_data


Epoch,0
_step,20
_runtime,10
_timestamp,1612994999
train batch loss,0.10994


Epoch,▁
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
_runtime,▁▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆███
_timestamp,▁▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆███
train batch loss,▃▃▄█▃▃▂▁▅▃▁▁▁▂▂▁▁▁▁▃


wandb: wandb version 0.10.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


walmart_amazon_exp_data
precision, recall, F1, support: (0.6880733944954128, 0.7772020725388601, 0.7299270072992701, None)





In [9]:
len(train_data)

384